# Справочники Cross Web
Пример ноутбука для работы со справочниками Cross Web.

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter 

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.crossweb import catalogs as cwc
from mediascope_api.crossweb import tasks as cwts


mnet = msnet.MediascopeApiNetwork()
cats = cwc.CrossWebCats()

# Доступные атрибуты для параметров

При формировании задания можно указать следующие параметры:

- **filters** - Фильтры
- **slices** - Срезы
- **statistics** - Статистики
- **scales** - Шкалы, если заданы статистики со шкалами (drfd, reachN)

Доступные атрибуты и допустимые для них значения можно получить методом `get_media_unit()`:

In [ ]:
cats.get_media_unit()

# Доступный период данных

Узнать доступный период данных можно после выполнения ячейки ниже

In [ ]:
cats.get_date_range()

# Типы пользования интернетом
Получить список типов пользования интернетом (usetype) можно после выполнения ячейки ниже

In [ ]:
cats.get_usetype()

# Социально-демографические и географические переменные

Получим список всех переменных, доступных в проекте Cross Web

In [ ]:
df_demo = cats.get_property()
df_demo

Экспорт справочника в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
demo_cat = cats.get_property()
writer = pd.ExcelWriter('demo_cat.xlsx')
demo_cat.to_excel(writer, 'demo_cat', index=False)
writer.save()

### Важно!
Demo и geo переменные, которые используются в заданиях, должны быть прописаны строго в следующем формате: 

__demoVariable__

То есть первое слово в переменной должно быть написано со строчной буквы, второе - с заглавной. Пример: __ageGroup__

### Поиск переменных

Поиск переменных осуществляется с помощью функции `cats.find_property()`, которая находит все переменные, содержащие заданный текст как в самом ее названии, так и в ее названии для API Cross Web (поля 'name' и 'entityTitle'). 

Если нужно найти переменную без входящих в нее категорий, то необходимо передать в эту функцию дополнительный параметр

```
    expand=False
```

Если нужно найти переменную с входящими в нее категориями, то можно не передавать параметр expand. В таком случае для него автоматически проставится значение True.

Найдем переменные, которые содержат текст "Пол"

In [ ]:
cats.find_property('Sex')

Рассмотрим географические переменные.

Численность населенного пункта (cityPop) и численность населенного пункта 100k+ или 100k- (cityPop100)

In [ ]:
cats.find_property('cityPop', expand=True)

Федеральный округ (federalOkrug)

In [ ]:
cats.find_property('federalOkrug', expand=True)

# Медиа справочник

Получим весь медиа справочник и отобразим первые пять записей.

### Важно!

**Загрузка всего медиа справочника может потребовать много памяти и времени. Для экономии памяти и ускорения загрузки, пожалуйста, пользуйтесь поиском по медиа справочнику.**

In [ ]:
df_media = cats.get_media()
df_media.head()

Экспорт медиа справочника в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
media_cat = cats.get_media()
writer = pd.ExcelWriter('media_cat.xlsx')
media_cat.to_excel(writer, 'media_cat', index=False)
writer.save()

**В Cross Web трехуровневая иерархия медиа объектов:**

- Холдинг (holding) = Холдинг
- Ресурс (resource) = Проект/Сайт
- Медиа продукт (product) = Сабсекция

Медиа продукт - это раздел в десктопном и мобильном вебе или приложение. Разделы в десктопном и мобильном вебе определяются регулярным выражением.

В проекте Cross Web определяется тематика (themeName) на уровне медиа продукта. Таким образом, разные разделы одного ресурса могут относиться к разным тематикам.

### Холдинги
Получим список холдингов

In [ ]:
df_holdings = cats.get_holding()
df_holdings

### Ресурсы
Получим список ресурсов и отобразим первые пять записей

In [ ]:
df_resources = cats.get_resource()
df_resources.head()

### Медиа продукты
Получим список медиа продуктов и отобразим первые 5 записей

In [ ]:
df_products = cats.get_product()
df_products.head()

### Тематики
Получим список тематик и отобразим первые 5 записей

In [ ]:
df_thematics = cats.get_theme()
df_thematics.head()

### Поиск в медиа справочнике

Поиск в медиа справочнике осуществляется с помощью следующих методов: 

- **get_media()** - отображает все поля медиа справочника; 
- **get_holding()** - отображает название холдинга и его id; 
- **get_resource()** - отображает название ресурса и его id; 
- **get_product()** - отображает название медиа продукта и его id; 
- **get_theme()** - отображает название тематики и ее id. 

Для всех перечисленных выше методов поддерживается поиск по следующим параметрам:

- **holding** - поиск по названию холдинга. Возможно задавать часть названия.
- **holding_ids** - поиск по cписку идентификаторов холдингов.
- **resource** - поиск по названию ресурса. Возможно задавать часть названия.
- **resource_ids** - поиск по cписку идентификаторов ресурсов.
- **product** - поиск по названию медиа продукта. Возможно задавать часть названия.
- **product_ids** - поиск по cписку идентификаторов медиа продуктов.
- **theme** - поиск по названию тематики. Возможно задавать часть названия.
- **theme_ids** - поиск по cписку идентификаторов тематик.


#### Поиск по названию
Найдем холдинг "Авито" в медиа справочнике и выведем всю информацию по его медиа объектам

In [ ]:
df_avito = cats.get_media(holding='Авито')
df_avito

#### Поиск по идентификатору
Найдем все медиа объекты с идентификаторами тематик
- id 20 - Почта
- id 42 - Работа

In [ ]:
df_media_by_id = cats.get_media(theme_ids=[20, 42])
df_media_by_id

#### Поиск по нескольким параметрам

Параметры поиска можно комбинировать. 

Добавим к предыдущему примеру еще один параметр поиска - название ресурса Avito. 

In [ ]:
df_media_by_id_resource = cats.get_media(theme_ids=[20, 42], resource='Avito')
df_media_by_id_resource